# Credit Card Fraud Detection
***
## Table of Contents
***

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pandas.api.types import is_numeric_dtype

## 1. Introduction


## 2. Loading Data
The dataset used in this project (retrieved from [Kaggle - Credit Card Fraud Detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud)) contains credit card transactions made by European cardholders during September 2013. It captures transaction activity over a two-day period, containing a total of 284,807 transactions, of which only 492 are fraudulent. This results in an extreme class imbalance, with fraudulent transactions representing just 0.172% of all records.

- **V1**, **V2**, ..., **V28**: These are the principal components derived from PCA, capturing the most significant variance in the original feature space.

- **Time**: Represents the number of seconds elapsed between each transaction and the first transaction in the dataset.

- **Amount**: Indicates the transaction amount, which is useful for cost-sensitive learning approaches.

- **Class**: The target variable, where $1$ denotes a fraudulent transaction and $0$ denotes a legitimate one.

In [2]:
df = pd.read_csv("_datasets/credit_card.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## References
1. Le Borgne, Y.-A., Siblini, W., Lebichot, B., & Bontempi, G. (2022). *Reproducible Machine Learning for Credit Card Fraud Detection - Practical Handbook*. Université Libre de Bruxelles. <br>
https://fraud-detection-handbook.github.io/fraud-detection-handbook/Foreword.html